The goal of the code below is to use topic modeling to investigate the relationship between some philosophical texts. In a previous step, I scraped them up from the Gutenberg project and stored them locally as text files.

To start building the topic model - load in some libraries, including numpy and sklearn.

In [1]:
import glob, sys, string
import numpy as np
from sklearn import feature_extraction, decomposition

Define some functions, where we'll fit the model and investigate the output.

In [2]:
def print_top_words(model, feature_names, n_top_words):
        for topic_idx, topic in enumerate(model.components_):
                print("Topic #%d:" % topic_idx)
                print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

def topic_modeling_prep(filenames, n_features):
        vectorizer = feature_extraction.text.CountVectorizer(input='filename', max_df=0.95, min_df=2, max_features=n_features, stop_words='english')
        tf_matrix = vectorizer.fit_transform(filenames)
        tf_array = tf_matrix.toarray()
        return vectorizer, tf_matrix, tf_array

def calc_lda(tf_array, n_topics=10,max_iter=100):
        lda = decomposition.LatentDirichletAllocation(n_topics=n_topics, max_iter=max_iter,
                                learning_method='online',
                                learning_offset=50.)
        doctopic = lda.fit_transform(tf_array)
        return doctopic, lda

def assign_topic(text_names,doctopic,num_topics):
        doctopic_grouped = np.zeros((text_names.shape[0], num_topics))
        for i, name in enumerate(text_names):
                doctopic_grouped[i, :] = np.mean(doctopic[text_names == name, :], axis=0)
        doc_topic_assign = np.argmax(doctopic_grouped, axis=1)
        return doc_topic_assign

def summarize_results(doc_topic_assign, lda, vectorizer, filenames):
        summary = read_summary()
        bookids = [f[5:-4] for f in filenames]

        # for each topic, print out words and documents.
        feature_names = vectorizer.get_feature_names()
        n_top_words = 5
        for topic_idx, topic in enumerate(lda.components_):
                print "Topic #%d:" % topic_idx,
                print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

                # which docs are placed in this topic?
                ids = np.where(doc_topic_assign==topic_idx)[0].tolist()
                print("\n".join([summary[(bookids[i],'title')] + " by " + summary[(bookids[i],'author')]  for i in ids]))
                print("\n")

def read_summary():
        # read in summary file with title/author/year info
        summary = {}
        with open('data/summary.txt','r') as f1:
                for line in f1:
                        cols                            = line.split('\t')
                        summary[(str(cols[0]),'title')] = str(cols[1])
                        summary[(str(cols[0]),'author')]= str(cols[2])
                        summary[(str(cols[0]),'year')]  = str(cols[3]).rstrip()
        return summary


Now let's run it! This step takes a few seconds. Then below is the output by topic.

In [3]:
# constants
num_topics = 10
num_features = 1000

# read input files
filenames = glob.glob("data/[0-9]*.txt")
text_names = np.asarray(filenames)

# create model
vectorizer, tf_matrix, tf_array = topic_modeling_prep(filenames,num_features)
doctopic, lda = calc_lda(tf_array,num_topics)
doc_topic_assign = assign_topic(text_names,doctopic,num_topics)

# describe results
summarize_results(doc_topic_assign, lda, vectorizer,filenames)


Topic #0: thou socrates god did soul
Apology by Plato
Euthyphro by Plato
The Queen's Matrimonial Ladder by William Hone
The Philosophy of Despair by David Starr Jordan
The Republic by Plato
Thus Spake Zarathustra by Friedrich Nietzsche
Ion by Plato
Beyond Good and Evil by Friedrich Nietzsche
The Case Of Wagner, Nietzsche Contra Wagner, and Selected Aphorisms. by Friedrich Nietzsche.
Bacon by Richard William Church
The Theological Tractates and The Consolation of Philosophy by Anicius Manlius Severinus Boethius
A Little Book of Stoicism by St. George Stock
Theaetetus by Plato


Topic #1: sec proposition propositions logic conclusion
Logic, Inductive and Deductive by William Minto
A Theological-Political Treatise [Part II] by Benedict of Spinoza
Symbolic Logic by Lewis Carroll
The Poetics by Aristotle
Logic by Carveth Read
The Categories by Aristotle
Sextus Empiricus and Greek Scepticism by Mary Mills Patrick
The Game of Logic by Lewis Carroll


Topic #2: laws cause causes cases small
A 

That's pretty cool! Some of the groups almost make sense! Is there a way to visualize this?

In [4]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [5]:
pyLDAvis.sklearn.prepare(lda,tf_matrix, vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      18.731421        1       1 -0.039866  0.070960
3      17.710737        1       2 -0.041638  0.019303
4      16.922979        1       3  0.055255  0.066583
5      12.472854        1       4  0.009948 -0.019954
0       7.661316        1       5  0.148349 -0.057594
7       5.838099        1       6  0.159664  0.013109
6       5.512489        1       7 -0.116050  0.037634
1       5.304795        1       8  0.005153  0.075432
9       5.144443        1       9 -0.130569 -0.003202
8       4.700868        1      10 -0.050247 -0.202273, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
422   Default  5847.000000          greek  5847.000000  30.0000  30.0000
544   Default  4823.000000         letter  4823.000000  29.0000  29.0000
859   Default  5628.000000          small  5628.000000  28.0000  28.0000
415   Default  6662.000000            god  6662.000000  27.0000  27.0000
420   Default  2572.000000     government  2572.000000  26.0000  26.0000
460   Default  5774.000000           idea  5774.000000  25.0000  25.0000
108   Default  4820.000000           body  4820.000000  24.0000  24.0000
931   Default  1643.000000           thou  1643.000000  23.0000  23.0000
669   Default  3446.000000         people  3446.000000  22.0000  22.0000
463   Default  5064.000000          ideas  5064.000000  21.0000  21.0000
754   Default  3095.000000    proposition  3095.000000  20.0000  20.0000
340   Default  5714.000000      existence  5714.000000  19.0000  19.0000
862   Default  2028.000000       socrates  2028.000000  18.0000  18.0000
630   Default  6152.000000         object  6152.000000  17.0000  17.0000
779   Default  3824.000000        reality  3824.000000  16.0000  16.0000
128   Default  6911.000000          cause  6911.000000  15.0000  15.0000
896   Default  2671.000000      substance  2671.000000  14.0000  14.0000
175   Default  3308.000000     conception  3308.000000  13.0000  13.0000
682   Default  3799.000000      phenomena  3799.000000  12.0000  12.0000
763   Default  2353.000000           pure  2353.000000  11.0000  11.0000
468   Default  1910.000000            iii  1910.000000  10.0000  10.0000
467   Default  2241.000000             ii  2241.000000   9.0000   9.0000
755   Default  2456.000000   propositions  2456.000000   8.0000   8.0000
564   Default  2813.000000           love  2813.000000   7.0000   7.0000
344   Default  6791.000000     experience  6791.000000   6.0000   6.0000
531   Default  5763.000000           laws  5763.000000   5.0000   5.0000
948   Default  2399.000000  understanding  2399.000000   4.0000   4.0000
829   Default  1276.000000            sec  1276.000000   3.0000   3.0000
860   Default  2906.000000         social  2906.000000   2.0000   2.0000
457   Default  6644.000000          human  6644.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
969   Topic10   231.338899            vii   558.960242   2.1752  -5.5323
25    Topic10   229.542360       adequate   566.354347   2.1543  -5.5401
658   Topic10   518.681749           pain  1281.197497   2.1532  -4.7249
863   Topic10   232.625971         solely   577.313731   2.1485  -5.5268
317   Topic10   632.798794        essence  1802.192534   2.0108  -4.5260
747   Topic10   765.587387          proof  2196.084456   2.0036  -4.3355
483   Topic10   207.407739       inasmuch   621.462565   1.9600  -5.6415
172   Topic10   497.375332       conceive  1542.926072   1.9253  -4.7668
173   Topic10   393.517467      conceived  1247.711710   1.9035  -5.0011
966   Topic10   197.559688             vi   631.807403   1.8949  -5.6901
671   Topic10   296.274892      perceived   992.158138   1.8488  -5.2849
665   Topic10   144.078633        passive   506.740020   1.7998  -6.0058
617   Topic10   480.2279